In [1]:
import pandas as pd
import os
import csv
import re

In [6]:
# set the directory path
dir_path = 'D:\\'

# create a new CSV file
with open('file_names.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # iterate through all the files in the directory and its subdirectories
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            # write the file name to the CSV file
            writer.writerow([os.path.join(root, file)])


In [7]:
df_tbu = pd.DataFrame(columns=['Details', 'QTY(PCS)', 'Unit Price', 'Amount', '日期', '主播', 'INV'])
invoice_regex = r".*(MC\d+)"
count = 0
# find the invoice number that starts with "MC" and is followed by numbers

# read the CSV file and get the first file name
file_names = pd.read_csv('file_names.csv', header=None)
for path in file_names.iloc[:, 0]:
    if path.endswith('.xlsx' or '.xls'):        
        if re.match(invoice_regex, path) is not None:
            file = path
            if "~$" in file:
                continue
            
       
    else:
        continue
    try:
        df = pd.read_excel(file, sheet_name="Invoice")                    
        col1=df.eq("Details").any(axis=0).idxmax()
        col2=df.eq("Amount").any(axis=0).idxmax()
        col3=df.eq("Unit Price").any(axis=0).idxmax()
        col4=df.eq("QTY(PCS)").any(axis=0).idxmax()
        df.dropna(subset=[col1, col2,col3, col4], inplace=True)
        df.dropna(axis=1, how='all', inplace=True)
        df.columns = df.iloc[0,]    
        df=df[['Details', 'QTY(PCS)', 'Unit Price', 'Amount', '日期', '主播']]
        df["INV"]=re.match(invoice_regex, file).groups()[0]
        df.drop(df.index[0], inplace=True)
        df_tbu = pd.concat([df_tbu,df], ignore_index=True, join="inner")
    except Exception as e:
        count += 1
        print (f"Error:{count} {file}: {e}")
        
    continue
df_tbu['日期'] = df_tbu['日期'].map(lambda x: pd.to_datetime(x,unit='D', origin=pd.Timestamp('1900-01-01')) if isinstance(x, int) else x)
df_tbu.to_csv('output.csv', index=False)




Error:1 test.png: [Errno 2] No such file or directory: 'test.png'
Error:2 test.png: [Errno 2] No such file or directory: 'test.png'
Error:3 test.png: [Errno 2] No such file or directory: 'test.png'
Error:4 test.png: [Errno 2] No such file or directory: 'test.png'
Error:5 test.png: [Errno 2] No such file or directory: 'test.png'
Error:6 test.png: [Errno 2] No such file or directory: 'test.png'
Error:7 test.png: [Errno 2] No such file or directory: 'test.png'
Error:8 test.png: [Errno 2] No such file or directory: 'test.png'
Error:9 test.png: [Errno 2] No such file or directory: 'test.png'
Error:10 test.png: [Errno 2] No such file or directory: 'test.png'
Error:11 test.png: [Errno 2] No such file or directory: 'test.png'
Error:12 test.png: [Errno 2] No such file or directory: 'test.png'
Error:13 test.png: [Errno 2] No such file or directory: 'test.png'
Error:14 test.png: [Errno 2] No such file or directory: 'test.png'
Error:15 test.png: [Errno 2] No such file or directory: 'test.png'
Erro